In [22]:
# test

!wget -qO /content/miniconda.sh https://repo.anaconda.com/miniconda/Miniconda3-latest-Linux-x86_64.sh
!bash /content/miniconda.sh -b -p /content/miniconda
!rm /content/miniconda.sh

!/content/miniconda/bin/conda create -y -n py311 python=3.11 pip

!/content/miniconda/bin/conda run -n py311 python -V

!/content/miniconda/bin/conda run -n py311 python -m pip install -r requirements.txt

!/content/miniconda/bin/conda run -n py311 python train.py --arg1 value1

PREFIX=/content/miniconda
Unpacking bootstrapper...
Unpacking payload...

Installing base environment...

Preparing transaction: ...working... done
Executing transaction: ...working... done
installation finished.
    You currently have a PYTHONPATH environment variable set. This may cause
    unexpected behavior when running the Python interpreter in Miniconda3.
    For best results, please verify that your PYTHONPATH only points to
    directories of packages that are compatible with the Python interpreter
    in Miniconda3: /content/miniconda
Jupyter detected...

CondaToSNonInteractiveError: Terms of Service have not been accepted for the following channels. Please accept or remove them before proceeding:
    - https://repo.anaconda.com/pkgs/main
    - https://repo.anaconda.com/pkgs/r

To accept these channels' Terms of Service, run the following commands:
    conda tos accept --override-channels --channel https://repo.anaconda.com/pkgs/main
    conda tos accept --override-channels -

## 네이버 쇼핑 리뷰를 보고 판매자 답변 자동 생성하기(정중/사과/보상 제안)

#### 데이터 출처 : https://github.com/bab2min/corpus

1. 라이브러리 설치

In [9]:
!python -m pip install -U pip
!python -m pip install "torch>=2.2" "transformers>=4.41" "datasets>=2.19" accelerate evaluate scikit-learn tqdm peft

2. MPS 확인(Apple Silicon GPU)

In [ ]:
import torch
print(torch.__version__)
print("mps available:", torch.backends.mps.is_available())

2.9.1
mps available: True


import os
os.environ["PYTORCH_ENABLE_MPS_FALLBACK"] = "1"

3. 데이터 파싱

In [11]:
from pathlib import Path
import random

RAW_PATH = Path("./naver_shopping.txt")

def load_reviews(min_len=5, max_n=None, seed=42):
    random.seed(seed)
    rows = []
    with RAW_PATH.open("r", encoding="utf-8") as f:
        for line in f:
            line = line.strip()
            if not line:
                continue
            parts = line.split("\t", 1)
            if len(parts) != 2:
                continue
            rating_s, text = parts
            try:
                rating = int(rating_s)
            except:
                continue
            text = text.strip()
            if len(text) < min_len:
                continue
            rows.append((rating, text))
    random.shuffle(rows)
    if max_n:
        rows = rows[:max_n]
    return rows

rows = load_reviews(max_n=30000)  # 8GB면 처음엔 3만 이하 추천
print("loaded:", len(rows))
print(rows[0])

loaded: 30000
(2, '냄새랑 맛이 생각보다 너무 역해요')


4. 불만 유형(키워드) 분류

In [12]:
COMPLAINT_RULES = {
    "배송": ["배송", "늦", "지연", "도착", "택배"],
    "품질/불량": ["불량", "하자", "고장", "깨", "찢", "누수", "작동", "불안정"],
    "포장": ["포장", "박스", "파손", "훼손", "완충"],
    "가격/가성비": ["비싸", "가격", "가성비", "싸", "할인"],
    "응대/서비스": ["응대", "문의", "연락", "고객센터", "불친절"],
}

def detect_topics(text: str):
    topics = []
    for topic, kws in COMPLAINT_RULES.items():
        if any(kw in text for kw in kws):
            topics.append(topic)
    return topics[:2]  # 너무 많으면 1~2개만

5. 템플릿 기반 판매자 답변 생성

In [13]:
def make_seller_reply(rating: int, review: str) -> str:
    topics = detect_topics(review)
    topic_phrase = " / ".join(topics) if topics else None

    # 보상 제안(너무 구체적 금액은 피하고 옵션만 제시)
    compensation = random.choice([
        "교환 또는 환불을 도와드리겠습니다.",
        "확인 후 쿠폰/부분환불 등 가능한 보상안을 안내드리겠습니다.",
        "불편을 줄이기 위해 교환/환불 절차를 빠르게 진행해드리겠습니다."
    ])

    # 긍정(4~5)
    if rating >= 4:
        extra = "앞으로도 더 좋은 상품과 서비스로 보답하겠습니다."
        upsell = random.choice([
            "재구매해주시면 감사하겠습니다.",
            "다음에도 만족스러운 경험을 드리겠습니다.",
            "소중한 후기 감사합니다."
        ])
        if topic_phrase:
            return f"안녕하세요, 고객님. {topic_phrase} 관련하여 만족하셨다니 정말 기쁩니다. {extra} {upsell}"
        return f"안녕하세요, 고객님. 소중한 후기 감사합니다. {extra} {upsell}"

    # 부정(1~2)
    if rating <= 2:
        apology = "불편을 드려 진심으로 죄송합니다."
        ask = "주문 정보와 문제 상황을 확인할 수 있도록 문의 남겨주시면 빠르게 도와드리겠습니다."
        if topic_phrase:
            return f"안녕하세요, 고객님. {topic_phrase} 관련하여 {apology} {ask} {compensation}"
        return f"안녕하세요, 고객님. {apology} {ask} {compensation}"

    # 중립(3) — 과제에서는 제외해도 되고, 남겨도 됨
    neutral = "의견 남겨주셔서 감사합니다."
    improve = "말씀해주신 부분은 개선하여 더 나은 서비스로 보답하겠습니다."
    if topic_phrase:
        return f"안녕하세요, 고객님. {topic_phrase} 관련하여 {neutral} {improve}"
    return f"안녕하세요, 고객님. {neutral} {improve}"

# 샘플 확인
for r, t in rows[:5]:
    print("RATING:", r)
    print("REVIEW:", t)
    print("REPLY:", make_seller_reply(r, t))
    print("-"*80)

RATING: 2
REVIEW: 냄새랑 맛이 생각보다 너무 역해요
REPLY: 안녕하세요, 고객님. 불편을 드려 진심으로 죄송합니다. 주문 정보와 문제 상황을 확인할 수 있도록 문의 남겨주시면 빠르게 도와드리겠습니다. 확인 후 쿠폰/부분환불 등 가능한 보상안을 안내드리겠습니다.
--------------------------------------------------------------------------------
RATING: 5
REVIEW: 진짜대박좋아요만족^^
REPLY: 안녕하세요, 고객님. 소중한 후기 감사합니다. 앞으로도 더 좋은 상품과 서비스로 보답하겠습니다. 다음에도 만족스러운 경험을 드리겠습니다.
--------------------------------------------------------------------------------
RATING: 5
REVIEW: 좋아요 배송도 빠르고 가겨도 마니 저렴하고~~^^
REPLY: 안녕하세요, 고객님. 배송 관련하여 만족하셨다니 정말 기쁩니다. 앞으로도 더 좋은 상품과 서비스로 보답하겠습니다. 다음에도 만족스러운 경험을 드리겠습니다.
--------------------------------------------------------------------------------
RATING: 1
REVIEW: 1년 넘게 쓴 와이퍼보다 능력이 떨어지내요. 워셔액도 못 닦아내네요. 이런거 팔지 않으셨으면 좋겠네요
REPLY: 안녕하세요, 고객님. 불편을 드려 진심으로 죄송합니다. 주문 정보와 문제 상황을 확인할 수 있도록 문의 남겨주시면 빠르게 도와드리겠습니다. 확인 후 쿠폰/부분환불 등 가능한 보상안을 안내드리겠습니다.
--------------------------------------------------------------------------------
RATING: 2
REVIEW: 진짜뚱뚱하신분만사셔야될듯
REPLY: 안녕하세요, 고객님. 불편을 드려 진심으로 죄송합니다. 주문

6. 학습 데이터 구성 + split

In [14]:
from sklearn.model_selection import train_test_split

def build_example(rating, review):
    # 프롬프트: 역할/규칙/입력
    prompt = (
        "### 역할: 당신은 온라인 쇼핑몰 판매자입니다.\n"
        "### 규칙:\n"
        "- 한국어 존댓말로 정중하게 작성\n"
        "- 2~4문장으로 간결하게\n"
        "- 부정 리뷰(1~2점)에는 사과+해결책+보상(쿠폰/교환/환불 중 하나) 포함\n"
        "- 긍정 리뷰(4~5점)에는 감사+재구매/재방문 유도 포함\n"
        "### 고객 리뷰:\n"
        f"{review}\n"
        "### 판매자 답변:\n"
    )
    completion = make_seller_reply(rating, review)
    return {"prompt": prompt, "completion": completion, "rating": rating, "review": review}

# 3점은 빼서 “명확한” 긍/부정만 학습(추천)
filtered = [(r, t) for (r, t) in rows if r in (1,2,4,5)]
data = [build_example(r, t) for r, t in filtered[:20000]]  # 최대 2만개만

train_data, valid_data = train_test_split(data, test_size=0.05, random_state=42)

print(len(train_data), len(valid_data))
print(train_data[0]["prompt"])
print("->", train_data[0]["completion"])

19000 1000
### 역할: 당신은 온라인 쇼핑몰 판매자입니다.
### 규칙:
- 한국어 존댓말로 정중하게 작성
- 2~4문장으로 간결하게
- 부정 리뷰(1~2점)에는 사과+해결책+보상(쿠폰/교환/환불 중 하나) 포함
- 긍정 리뷰(4~5점)에는 감사+재구매/재방문 유도 포함
### 고객 리뷰:
저렴하게 좋은 제품을 구매한거 같아 만족해요^^
### 판매자 답변:

-> 안녕하세요, 고객님. 소중한 후기 감사합니다. 앞으로도 더 좋은 상품과 서비스로 보답하겠습니다. 재구매해주시면 감사하겠습니다.


7. 모델/토크나이저 로드 + LoRA 적용

In [15]:
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM
from peft import LoraConfig, get_peft_model

MODEL_ID = "skt/kogpt2-base-v2"   # 안 되면 여기만 다른 KoGPT2 repo로 변경
device = "mps" if torch.backends.mps.is_available() else "cpu"
print("device:", device)

tokenizer = AutoTokenizer.from_pretrained(MODEL_ID, use_fast=True)

# GPT2류는 pad_token이 없는 경우가 많아서 eos로 맞추는 게 안전
if tokenizer.pad_token is None:
    tokenizer.pad_token = tokenizer.eos_token

model = AutoModelForCausalLM.from_pretrained(MODEL_ID)
model.config.use_cache = False
model.gradient_checkpointing_enable()  # 메모리 절약

# LoRA 설정 (GPT-2 계열에서 흔히 쓰는 target)
lora_config = LoraConfig(
    r=8,
    lora_alpha=16,
    lora_dropout=0.05,
    bias="none",
    task_type="CAUSAL_LM",
    target_modules=["c_attn", "c_proj"],  # GPT-2 계열에 흔한 모듈명
    fan_in_fan_out=True
)

model = get_peft_model(model, lora_config)
model.print_trainable_parameters()

model.to(device)

device: cpu


/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

pytorch_model.bin:   0%|          | 0.00/513M [00:00<?, ?B/s]

trainable params: 811,008 || all params: 125,975,040 || trainable%: 0.6438


PeftModelForCausalLM(
  (base_model): LoraModel(
    (model): GPT2LMHeadModel(
      (transformer): GPT2Model(
        (wte): Embedding(51200, 768)
        (wpe): Embedding(1024, 768)
        (drop): Dropout(p=0.1, inplace=False)
        (h): ModuleList(
          (0-11): 12 x GPT2Block(
            (ln_1): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
            (attn): GPT2Attention(
              (c_attn): lora.Linear(
                (base_layer): Conv1D(nf=2304, nx=768)
                (lora_dropout): ModuleDict(
                  (default): Dropout(p=0.05, inplace=False)
                )
                (lora_A): ModuleDict(
                  (default): Linear(in_features=768, out_features=8, bias=False)
                )
                (lora_B): ModuleDict(
                  (default): Linear(in_features=8, out_features=2304, bias=False)
                )
                (lora_embedding_A): ParameterDict()
                (lora_embedding_B): ParameterDict()
          

8. Dataset 만들기 + 토크나이징

In [16]:
from datasets import Dataset

def pack_text(ex):
    return ex["prompt"] + ex["completion"]

train_ds = Dataset.from_dict({"text": [pack_text(x) for x in train_data]})
valid_ds = Dataset.from_dict({"text": [pack_text(x) for x in valid_data]})

MAX_LEN = 192  # 8GB 추천(256도 가능하지만 터질 수 있음)

def tok(batch):
    return tokenizer(
        batch["text"],
        truncation=True,
        max_length=MAX_LEN,
        padding="max_length"
    )

train_tok = train_ds.map(tok, batched=True, remove_columns=["text"])
valid_tok = valid_ds.map(tok, batched=True, remove_columns=["text"])


Map:   0%|          | 0/19000 [00:00<?, ? examples/s]

Map:   0%|          | 0/1000 [00:00<?, ? examples/s]

9. Trainer로 학습

In [17]:
from transformers import TrainingArguments, Trainer, DataCollatorForLanguageModeling

collator = DataCollatorForLanguageModeling(tokenizer=tokenizer, mlm=False)

args = TrainingArguments(
    output_dir="outputs/seller_reply_lora",
    per_device_train_batch_size=1,
    per_device_eval_batch_size=1,
    gradient_accumulation_steps=32,   # 배치 늘린 효과
    learning_rate=2e-4,              # LoRA는 보통 LR을 조금 높게
    num_train_epochs=1,
    logging_steps=50,
    eval_strategy="steps",
    eval_steps=200,
    save_steps=200,
    save_total_limit=2,
    report_to="none",
)

trainer = Trainer(
    model=model,
    args=args,
    train_dataset=train_tok,
    eval_dataset=valid_tok,
    data_collator=collator,
)

trainer.train()
trainer.save_model("outputs/seller_reply_lora")
tokenizer.save_pretrained("outputs/seller_reply_lora")


/usr/local/lib/python3.12/dist-packages/torch/utils/data/dataloader.py:668: UserWarning: 'pin_memory' argument is set as true but no accelerator is found, then device pinned memory won't be used.
  warnings.warn(warn_msg)


IndexError: index out of range in self

10. 생성 함수

In [ ]:
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer

FT_DIR = "outputs/seller_reply_lora"

ft_tokenizer = AutoTokenizer.from_pretrained(FT_DIR, use_fast=True)
if ft_tokenizer.pad_token is None:
    ft_tokenizer.pad_token = ft_tokenizer.eos_token

ft_model = AutoModelForCausalLM.from_pretrained(FT_DIR).to(device)
ft_model.config.use_cache = False

@torch.no_grad()
def generate_reply(prompt, max_new_tokens=80, temperature=0.7):
    inputs = ft_tokenizer(prompt, return_tensors="pt").to(device)
    out = ft_model.generate(
        **inputs,
        max_new_tokens=max_new_tokens,
        do_sample=True,
        temperature=temperature,
        top_p=0.9,
        eos_token_id=ft_tokenizer.eos_token_id,
        pad_token_id=ft_tokenizer.pad_token_id,
    )
    text = ft_tokenizer.decode(out[0], skip_special_tokens=True)
    # prompt 이후만 출력
    if prompt in text:
        return text.split(prompt, 1)[1].strip()
    return text.strip()


11. 프롬프트 3종 비교(단순/조건/강한 제약)

In [ ]:
sample_review_neg = "배송이 너무 늦고 포장도 찢어져서 왔어요. 정말 실망입니다."
sample_review_pos = "배송도 빠르고 제품 품질이 좋아요. 재구매 의사 있습니다!"

def prompt_basic(review):
    return f"리뷰: {review}\n판매자 답변:"

def prompt_with_rules(review, neg=True):
    rules = (
        "규칙: 한국어 존댓말, 2~4문장.\n"
        + ("사과+해결책+보상(쿠폰/교환/환불 중 하나)을 반드시 포함.\n" if neg else "감사+재구매 유도 포함.\n")
    )
    return f"{rules}리뷰: {review}\n판매자 답변:"

def prompt_project_style(review):
    # 학습 때 쓰던 포맷(일관성 최고)
    return (
        "### 역할: 당신은 온라인 쇼핑몰 판매자입니다.\n"
        "### 규칙:\n"
        "- 한국어 존댓말로 정중하게 작성\n"
        "- 2~4문장으로 간결하게\n"
        "- 부정 리뷰에는 사과+해결책+보상 제안 포함\n"
        "- 긍정 리뷰에는 감사+재구매 유도 포함\n"
        "### 고객 리뷰:\n"
        f"{review}\n"
        "### 판매자 답변:\n"
    )

for p in [prompt_basic(sample_review_neg),
          prompt_with_rules(sample_review_neg, neg=True),
          prompt_project_style(sample_review_neg)]:
    print("PROMPT:\n", p)
    print("OUTPUT:\n", generate_reply(p))
    print("="*80)

for p in [prompt_basic(sample_review_pos),
          prompt_with_rules(sample_review_pos, neg=False),
          prompt_project_style(sample_review_pos)]:
    print("PROMPT:\n", p)
    print("OUTPUT:\n", generate_reply(p))
    print("="*80)


12. 규칙 준수 점수(간단 룰 기반)

In [ ]:
import re

def score_reply(review_rating, reply):
    score = 0
    # 문장 수(대충 .?! 기준)
    sentences = [s for s in re.split(r"[.?!]\s*", reply) if s.strip()]
    if 2 <= len(sentences) <= 4:
        score += 1

    if review_rating <= 2:
        if any(k in reply for k in ["죄송", "사과"]):
            score += 1
        if any(k in reply for k in ["교환", "환불", "쿠폰", "보상"]):
            score += 1
    if review_rating >= 4:
        if any(k in reply for k in ["감사", "고맙"]):
            score += 1
        if any(k in reply for k in ["재구매", "다음에도", "또 이용"]):
            score += 1
    return score

# 샘플 50개로 점수 보기
samples = random.sample(filtered, 50)
total = 0
for r, review in samples:
    p = prompt_project_style(review)
    out = generate_reply(p)
    total += score_reply(r, out)

print("avg rule score:", total/50)
